In [14]:
import natasha

In [1]:
import time
import torch
import textwrap
import numpy as np
import pandas as pd
from transformers import pipeline
from transformers import MarianMTModel, MarianTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import MBartTokenizer, MBartForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import Trainer, TrainingArguments

from datasets import load_dataset

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
ru_en_model_name = "Helsinki-NLP/opus-mt-ru-en"
en_ru_model_name = "Helsinki-NLP/opus-mt-en-ru"

ru_en_tokenizer = MarianTokenizer.from_pretrained(ru_en_model_name)
ru_en_model = MarianMTModel.from_pretrained(ru_en_model_name).to(device)

en_ru_tokenizer = MarianTokenizer.from_pretrained(en_ru_model_name)
en_ru_model = MarianMTModel.from_pretrained(en_ru_model_name).to(device)

In [5]:
def translate(text, model, tokenizer, device):
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
    translated_ids = model.generate(inputs["input_ids"])
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    print(translated_text)
    return translated_text

In [6]:
def use_model(ru_text, model, tokenizer, device):
    en_text = translate(ru_text, ru_en_model, ru_en_tokenizer, device)

    print(f"EN: {en_text}")
    
    inputs = tokenizer(en_text, return_tensors="pt", max_length=512, truncation=True).to(device)

    summary_ids = model.generate(inputs["input_ids"])

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    print(f"ENs: {summary}")

    # summary_words = " ".join(summary.split())
    
    ru_summary = translate(summary, en_ru_model, en_ru_tokenizer, device)

    return ru_summary

### BART/MT5/T5 (pipeline)

In [ ]:
summarizer_bart = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
summarizer_mt5 = pipeline("summarization", model="google/mt5-large", device=device)
summarizer_t5 = pipeline("summarization", model="t5-small", device=device)

In [7]:
# article_text = "Жизненные трудности поспособствовали моему увольнению, так как нужно идти на другую работу"
article_text = "Нарушение ТК РФ руководством в части организации совещаний в обеденное время и после окончания рабочего дня. Севещения для подготовки к совещаниям и совещания по итогам совещаний"
article_text = "Руководитель обозначает невыполнимый ежедневный план, не предоставив обучения и наставников для того чтобы разобраться в поставленной задаче. По причине невыполнения работы не отпустил к врачу"

print("Russian")

summary = summarizer_bart(article_text, max_length=5, min_length=2, do_sample=False, num_beams=5, length_penalty=3.0)
print(summary[0]['summary_text'])

summary = summarizer_mt5(article_text, max_length=5, min_length=2, do_sample=False, num_beams=5, length_penalty=3.0)
print(summary[0]['summary_text'])

summary = summarizer_t5(article_text, max_length=5, min_length=2, do_sample=False, num_beams=5, length_penalty=3.0)
print(summary[0]['summary_text'])

print("English")

article_text = translate(article_text, ru_en_model, ru_en_tokenizer, device)

summary = summarizer_bart(article_text, max_length=5, min_length=2, do_sample=False, num_beams=5, length_penalty=3.0)
print(translate(summary[0]['summary_text'], en_ru_model, en_ru_tokenizer, device))

summary = summarizer_mt5(article_text, max_length=5, min_length=2, do_sample=False, num_beams=5, length_penalty=3.0)
print(translate(summary[0]['summary_text'], en_ru_model, en_ru_tokenizer, device))

summary = summarizer_t5(article_text, max_length=5, min_length=2, do_sample=False, num_beams=5, length_penalty=3.0)
print(translate(summary[0]['summary_text'], en_ru_model, en_ru_tokenizer, device))

Russian

<extra_id_0>. <extra_id_1>.
уко
English
The manager designates an impossible daily plan without providing training and mentors to understand the task.
Менеджер
Менеджер
<extra_id_0> обозначает
<extra_id_0> обозначает
Менеджер назначает невозможное
Менеджер назначает невозможное


### MT5 (pure)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-large").to(device)

In [21]:
article_text = "Жизненные трудности поспособствовали моему увольнению, так как нужно идти на другую работу"
article_text = "Нарушение ТК РФ руководством в части организации совещаний в обеденное время и после окончания рабочего дня. Севещения для подготовки к совещаниям и совещания по итогам совещаний"
article_text = "Руководитель обозначает невыполнимый ежедневный план, не предоставив обучения и наставников для того чтобы разобраться в поставленной задаче. По причине невыполнения работы не отпустил к врачу"

use_model(article_text, model, tokenizer, device)

EN: Life's difficulties helped me quit because I had to go to another job.
ENs: <extra_id_0> me. <extra_id_1> I started a new job.  <extra_id_2> me.  <extra_id_3> me


'<extra_id_0> я. <extra_id_1> Я начал новую работу. <extra_id_2> я. <extra_id_3>'

### Pegasus XSUM

In [3]:
pegasus_tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
pegasus_model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
article_text = "Жизненные трудности поспособствовали моему увольнению, так как нужно идти на другую работу"
# article_text = "Нарушение ТК РФ руководством в части организации совещаний в обеденное время и после окончания рабочего дня. Севещения для подготовки к совещаниям и совещания по итогам совещаний"
# article_text = "Руководитель обозначает невыполнимый ежедневный план, не предоставив обучения и наставников для того чтобы разобраться в поставленной задаче. По причине невыполнения работы не отпустил к врачу"

article_text = translate(article_text, ru_en_model, ru_en_tokenizer, device)

inputs = pegasus_tokenizer(article_text, return_tensors="pt", max_length=512, truncation=True)

summary_ids = pegasus_model.generate(
    inputs["input_ids"],
    max_length=6,    # Ограничение по количеству токенов (приближенно 2-3 слова)
    min_length=2,    # Минимум 2 токена
    num_beams=5,     # Используем несколько лучей для лучшего результата
    length_penalty=3.0,  # Штраф за длинные тексты
    early_stopping=True
)

summary = pegasus_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(translate(summary, en_ru_model, en_ru_tokenizer, device))

Life's difficulties helped me quit because I had to go to another job.
Меня зовут Джон.
Меня зовут Джон.


In [12]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, pipeline

# Загружаем модель и токенизатор Pegasus
model_name = "google/pegasus-xsum"
pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Пайплайн для перевода (если нужно обработать русский текст)
translator_ru_en = pipeline("translation", model="Helsinki-NLP/opus-mt-ru-en")
translator_en_ru = pipeline("translation", model="Helsinki-NLP/opus-mt-en-ru")

# Текст на русском языке
article_text = "Жизненные трудности поспособствовали моему увольнению, так как нужно идти на другую работу."

# Переводим текст с русского на английский
translated_text = translator_ru_en(article_text)[0]['translation_text']
print("Переведенный текст на английский:", translated_text)

# Токенизация текста для модели Pegasus
inputs = pegasus_tokenizer(translated_text, return_tensors="pt", max_length=512, truncation=True)

# Генерация суммаризации с жестким ограничением на длину
summary_ids = pegasus_model.generate(
    inputs["input_ids"],
    max_length=6,    # Ограничение по количеству токенов (приближенно 2-3 слова)
    min_length=2,    # Минимум 2 токена
    num_beams=5,     # Используем несколько лучей для лучшего результата
    length_penalty=3.0,  # Штраф за длинные тексты
    early_stopping=True
)

# Декодирование суммаризации
summary = pegasus_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Summary in English:", summary)

# Переводим суммаризацию обратно на русский
final_summary = translator_en_ru(summary)[0]['translation_text']
print("Итоговая суммаризация на русском:", final_summary)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Переведенный текст на английский: Life's difficulties contributed to my dismissal because I had to go to another job.
Summary in English: I was sacked from
Итоговая суммаризация на русском: Меня уволили.


### Fine tuning of XSUM

In [2]:
model_name = "google/pegasus-xsum"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
dataset = load_dataset('cnn_dailymail', '3.0.0', split='train')

In [5]:
def preprocess_data(examples):
    inputs = examples['article']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'], max_length=128, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [6]:
tokenized_dataset = dataset.map(preprocess_data, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
    fp16=True,
    logging_dir='./logs',
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
model.save_pretrained("./fine_tuned_pegasus")
tokenizer.save_pretrained("./fine_tuned_pegasus")

In [ ]:
fine_tuned_model_dir = "./fine_tuned_pegasus"

tokenizer = PegasusTokenizer.from_pretrained(fine_tuned_model_dir)
model = PegasusForConditionalGeneration.from_pretrained(fine_tuned_model_dir)

article_text = "Жизненные трудности поспособствовали моему увольнению, так как нужно идти на другую работу."

inputs = tokenizer(article_text, return_tensors="pt", max_length=512, truncation=True)

summary_ids = model.generate(
    inputs["input_ids"],
    max_length=6,
    min_length=2,
    num_beams=5,
    length_penalty=3.0,
    early_stopping=True
)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Итоговая суммаризация:", summary)

### MBART (gazeta)

In [12]:
tokenizer = MBartTokenizer.from_pretrained("IlyaGusev/mbart_ru_sum_gazeta")
model = MBartForConditionalGeneration.from_pretrained("IlyaGusev/mbart_ru_sum_gazeta")

In [13]:
article_text = "Жизненные трудности поспособствовали моему увольнению, так как нужно идти на другую работу"
article_text = "Нарушение ТК РФ руководством в части организации совещаний в обеденное время и после окончания рабочего дня. Севещения для подготовки к совещаниям и совещания по итогам совещаний"
article_text = "Руководитель обозначает невыполнимый ежедневный план, не предоставив обучения и наставников для того чтобы разобраться в поставленной задаче. По причине невыполнения работы не отпустил к врачу"

input_ids = tokenizer([article_text], max_length=256,
                      padding="max_length", truncation=True,
                      return_tensors="pt")["input_ids"]

output_ids = model.generate(input_ids=input_ids,
                            no_repeat_ngram_size=4)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)
print(summary)

Руководитель обозначает невыполнимый ежедневный план, не предоставив обучения и наставников для того чтобы разобраться в поставленной задаче. По причине невыполнения работы не отпустил к врачу.


In [7]:
import gc
gc.collect()
torch.cuda.empty_cache()